In [1]:
import pandas as pd
import numpy as np
import torch
from tdc.single_pred.adme import ADME
from tdc import Evaluator
from tqdm.notebook import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader
from rdkit import Chem
from rdkit.Chem import AllChem
from matplotlib import pyplot as plt
from IPython import display

from typing import List, Tuple


class Featurizer:
    def __init__(self, y_column, smiles_col='Drug', **kwargs):
        self.y_column = y_column
        self.smiles_col = smiles_col
        self.__dict__.update(kwargs)
    
    def __call__(self, df):
        raise NotImplementedError()


class ECFPFeaturizer(Featurizer):
    def __init__(self, y_column, radius=2, length=1024, **kwargs):
        self.radius = radius
        self.length = length
        super().__init__(y_column, **kwargs)
    
    def __call__(self, df):
        fingerprints = []
        labels = []
        for i, row in df.iterrows():
            y = row[self.y_column]
            smiles = row[self.smiles_col]
            mol = Chem.MolFromSmiles(smiles)
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, self.radius, nBits=self.length)
            fingerprints.append(fp)
            labels.append(y)
        fingerprints = np.array(fingerprints)
        labels = np.array(labels)
        return fingerprints, labels

data = ADME('Solubility_AqSolDB')
split = data.get_split()
rmse = Evaluator(name = 'RMSE')

featurizer = ECFPFeaturizer(y_column='Y')
X_train, y_train = featurizer(split['train'])
X_valid, y_valid = featurizer(split['valid'])
X_test, y_test = featurizer(split['test'])

Downloading...
100%|██████████| 853k/853k [00:01<00:00, 475kiB/s] 
Loading...
Done!
[01:47:33] WARNING: not removing hydrogen atom without neighbors
[01:47:33] WARNING: not removing hydrogen atom without neighbors
[01:47:33] WARNING: not removing hydrogen atom without neighbors
[01:47:33] WARNING: not removing hydrogen atom without neighbors
[01:47:34] WARNING: not removing hydrogen atom without neighbors
[01:47:34] WARNING: not removing hydrogen atom without neighbors
[01:47:34] WARNING: not removing hydrogen atom without neighbors
[01:47:34] WARNING: not removing hydrogen atom without neighbors
[01:47:34] WARNING: not removing hydrogen atom without neighbors
[01:47:34] WARNING: not removing hydrogen atom without neighbors
[01:47:34] WARNING: not removing hydrogen atom without neighbors
[01:47:34] WARNING: not removing hydrogen atom without neighbors
[01:47:34] WARNING: not removing hydrogen atom without neighbors
[01:47:34] WARNING: not removing hydrogen atom without neighbors
[01:47

In [2]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise ValueError("input {0} not in allowable set{1}:".format(
            x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))


def one_of_k_encoding_unk(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


from rdkit.Chem import rdMolDescriptors

class GraphFeaturizer(Featurizer):
    def __call__(self, df):
        graphs = []
        labels = []
        for i, row in df.iterrows():
            y = row[self.y_column]
            smiles = row[self.smiles_col]
            mol = Chem.MolFromSmiles(smiles)
            
            edges = []
            for bond in mol.GetBonds():
                begin = bond.GetBeginAtomIdx()
                end = bond.GetEndAtomIdx()
                edges.append((begin, end))  # TODO: Add edges in both directions
            edges = np.array(edges)
            
            nodes = []
            for atom in mol.GetAtoms():
                # print(atom.GetAtomicNum(), atom.GetNumImplicitHs(), atom.GetTotalNumHs(), atom.GetSymbol(), atom.GetNumExplicitHs(), atom.GetTotalValence())
                results = one_of_k_encoding_unk(atom.GetAtomicNum(), range(11)) + one_of_k_encoding(
                    atom.GetDegree(), range(11)
                ) + one_of_k_encoding_unk(
                    atom.GetImplicitValence(), range(11)
                ) + [atom.GetIsAromatic()] + one_of_k_encoding_unk(
                    atom.GetTotalNumHs(), range(11)
                ) + [atom.GetNumImplicitHs(), atom.GetFormalCharge(), atom.GetNumRadicalElectrons(), atom.GetIsAromatic()] # TODO: Add atom features as a list, you can use one_of_k_encodings defined above
                # print(results)
                nodes.append(results)
            nodes = np.array(nodes)
            
            graphs.append((nodes, edges.T))
            labels.append(y)
        labels = np.array(labels)
        return [Data(
            x=torch.FloatTensor(x), 
            edge_index=torch.LongTensor(edge_index), 
            y=torch.FloatTensor([y])
        ) for ((x, edge_index), y) in zip(graphs, labels)]

In [3]:
featurizer = GraphFeaturizer('Y')
graph = featurizer(split['test'].iloc[:1])[0]

In [4]:
from torch_geometric.loader import DataLoader as GraphDataLoader


# prepare data loaders
batch_size = 64
train_loader = GraphDataLoader(featurizer(split['train']), batch_size=batch_size, shuffle=True)
valid_loader = GraphDataLoader(featurizer(split['valid']), batch_size=batch_size)
test_loader = GraphDataLoader(featurizer(split['test']), batch_size=batch_size)

[01:47:46] WARNING: not removing hydrogen atom without neighbors
[01:47:46] WARNING: not removing hydrogen atom without neighbors
[01:47:46] WARNING: not removing hydrogen atom without neighbors
[01:47:46] WARNING: not removing hydrogen atom without neighbors
[01:47:46] WARNING: not removing hydrogen atom without neighbors
[01:47:46] WARNING: not removing hydrogen atom without neighbors
[01:47:47] WARNING: not removing hydrogen atom without neighbors
[01:47:47] WARNING: not removing hydrogen atom without neighbors
[01:47:47] WARNING: not removing hydrogen atom without neighbors
[01:47:47] WARNING: not removing hydrogen atom without neighbors
[01:47:47] WARNING: not removing hydrogen atom without neighbors
[01:47:47] WARNING: not removing hydrogen atom without neighbors
[01:47:47] WARNING: not removing hydrogen atom without neighbors
[01:47:47] WARNING: not removing hydrogen atom without neighbors
[01:47:47] WARNING: not removing hydrogen atom without neighbors
[01:47:47] WARNING: not r

In [27]:
t1 = torch.ones((10, 2))
t2 = torch.zeros((10, 3))

torch.softmax(t1, dim=-1)

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]])

In [65]:
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool as gap

class MyAttentionModule(torch.nn.Module): # zakladamy ze atom ma 49 featerow
    def __init__(self, groupFeatures=1):
        super().__init__()
        self.conv = GCNConv(49, 49) # dla zebrania informacji od sasiadow
        self.gates = { # do wyliczenia atencji dla kazdej grupy cech - jest ich 9
            'AtomicNum': torch.nn.Linear(11, 1),
            'Degree': torch.nn.Linear(11, 1),
            'ImplicitValence': torch.nn.Linear(11, 1),
            'IsAromatic': torch.nn.Linear(1, 1),
            'TotalNumHs': torch.nn.Linear(11, 1),
            'NumImplicitHs': torch.nn.Linear(1, 1),
            'FormalCharge': torch.nn.Linear(1, 1),
            'NumRadicalElectrons': torch.nn.Linear(1, 1),
            'IsAromatic': torch.nn.Linear(1, 1)
        }
        
        self.feats = { # do transformacji grupy cech w wektor, na razie dziala tylko dla groupFeatures=1
            'AtomicNum': torch.nn.Linear(11, groupFeatures),
            'Degree': torch.nn.Linear(11, groupFeatures),
            'ImplicitValence': torch.nn.Linear(11, groupFeatures),
            'IsAromatic': torch.nn.Linear(1, groupFeatures),
            'TotalNumHs': torch.nn.Linear(11, groupFeatures),
            'NumImplicitHs': torch.nn.Linear(1, groupFeatures),
            'FormalCharge': torch.nn.Linear(1, groupFeatures),
            'NumRadicalElectrons': torch.nn.Linear(1, groupFeatures),
            'IsAromatic': torch.nn.Linear(1, groupFeatures)
        }

    def forward(self, x, edge_index, batch):
        x = self.conv(x, edge_index)
        # print(x.shape)
        subgroups = []
        subgroups.append(self.gates['AtomicNum'](x[:,0:11]))
        subgroups.append(self.gates['Degree'](x[:,11:22]))
        subgroups.append(self.gates['ImplicitValence'](x[:,22:33]))
        subgroups.append(self.gates['IsAromatic'](x[:,33:34]))
        subgroups.append(self.gates['TotalNumHs'](x[:,34:45]))
        subgroups.append(self.gates['NumImplicitHs'](x[:,45:46]))
        subgroups.append(self.gates['FormalCharge'](x[:,46:47]))
        subgroups.append(self.gates['NumRadicalElectrons'](x[:,47:48]))
        subgroups.append(self.gates['IsAromatic'](x[:,48:49]))
        logits = torch.cat(subgroups, dim=-1) # dla np x o ksztalcie (1200, 49) bedziemy mieli tensor istotnosci (1200, 9)
        logits = gap(logits, batch=batch) # nie pisal Pan o tym, ale chyba chcemy miec wektor atencji dla kazdej czasteczki a nie dla kazdego atomu z osobna, wiec usredniam dla czasteczki
        # czyli dla batch_size=64 bedziemy mieli pooling: (1200, 9) -> (64, 9)
        attention = torch.softmax(logits, dim=-1)
        
        subgroups = []
        subgroups.append(self.feats['AtomicNum'](x[:,0:11]))
        subgroups.append(self.feats['Degree'](x[:,11:22]))
        subgroups.append(self.feats['ImplicitValence'](x[:,22:33]))
        subgroups.append(self.feats['IsAromatic'](x[:,33:34]))
        subgroups.append(self.feats['TotalNumHs'](x[:,34:45]))
        subgroups.append(self.feats['NumImplicitHs'](x[:,45:46]))
        subgroups.append(self.feats['FormalCharge'](x[:,46:47]))
        subgroups.append(self.feats['NumRadicalElectrons'](x[:,47:48]))
        subgroups.append(self.feats['IsAromatic'](x[:,48:49]))
        x = torch.cat(subgroups, dim=-1) # kazda grupe przerzucamy przez warstwe liniowa i konkatenujemy: (1200, 49) -> (1200, 9)
        
        for i, atom_features in enumerate(x):
            x[i] *= attention[batch[i]] #przemnazamy grupy featerow przez ich istotnosci
        
        return x, attention

In [66]:
class GraphNeuralNetwork(torch.nn.Module):  # TODO: assign hyperparameters to attributes and define the forward pass
    def __init__(self, hidden_size, n_features=49, dropout=0.2):
        super().__init__()
        self.myAttentionModule = MyAttentionModule(1)
        self.conv1 = GCNConv(27, hidden_size)
        self.conv2 = GCNConv(hidden_size, int(hidden_size))
        self.conv3 = GCNConv(int(hidden_size), int(hidden_size))
        self.linear = torch.nn.Linear(int(hidden_size), 1)
        self.dropout = dropout
    
    def forward(self, x, edge_index, batch):
        x, att = self.myAttentionModule(x, edge_index, batch) #nie rozumiem czemu mialbym robic pooling, skoro nasz modul uzywamy na poczatku forward - jeszcze przed warstwami konwolucyjnymi
        #a chcemy moc pozniej uzyc nasza wyuczona warstwe do roznych innych datasetow i architektor, jak dobrze rozumiem
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        
        x = gap(x, batch)
        
        x = F.dropout(x, p=self.dropout, training=self.training)

        out = self.linear(x)

        return out, att

In [67]:
def train(train_loader, valid_loader):
    # hyperparameters definition
    hidden_size = 512
    epochs = 20
    learning_rate = 0.001
    
    # model preparation
    model = GraphNeuralNetwork(hidden_size)  # TODO: you can add more hyperparameters if needed
    model.train()
    
    # training loop
    optimizer = torch.optim.Adam(model.parameters(), learning_rate) # TODO: define an optimizer
    loss_fn = torch.nn.MSELoss()  # TODO: define a loss function
    for epoch in trange(1, epochs + 1, leave=False):
        for data in tqdm(train_loader, leave=False):
            x, edge_index, batch, y = data.x, data.edge_index, data.batch, data.y
            model.zero_grad()
            preds, att = model(x, edge_index, batch)
            loss = loss_fn(preds, y.reshape(-1, 1))
            loss.backward()
            optimizer.step()
    return model


def predict(model, test_loader):
    # evaluation loop
    preds_batches = []
    with torch.no_grad():
        for data in tqdm(test_loader):
            x, edge_index, batch = data.x, data.edge_index, data.batch
            
            preds, att = model(x, edge_index, batch)
            preds_batches.append(preds.cpu().detach().numpy())
            print(att)
    preds = np.concatenate(preds_batches)
    return preds


# training
model = train(train_loader, valid_loader)

# evaluation
predictions = predict(model, test_loader)

rmse_score = rmse(y_test, predictions.flatten())

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (27) must match the size of tensor b (9) at non-singleton dimension 0

In [58]:
print(f'RMSE = {rmse_score:.2f}')

RMSE = 1.64
